In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import pandas as pd
import os


from dash import Dash, dcc, html, Input, Output
import plotly.express as px



In [9]:
df= pd.read_excel(os.path.join('..', 'data', '20_08_24.xlsx'))
df

,CONSOLIDADO(S),ESPECIFICAÇÃO CRIME,MEIO EMPREGADO DEAC,MUNICÍPIO(S),BAIRRO(S),ANO DO FATO,MÊS DO FATO,RISP,AISP,DATA DO FATO,...,IDADE VÍTIMA,FAIXA ETÁRIA VÍTIMA,TIPO DE VÍTIMA,COR VÍTIMA,GRAU DE INSTRUÇÃO VÍTIMA,ESTADO CIVÍL VÍTIMA,SEXO AUTOR,IDADE AUTOR,COR AUTOR,GRAU DE RELACIONAMENTO
0,TRAFICO DE DROGAS,NaN,NaN,BELEM,GUAMA,2023,OUTUBRO,01ª RISP,05ª AISP,2023-10-19,...,88888,"PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS",P,66666.0,PARDA,NaN
1,LESAO CORPORAL,NaN,VIOLENCIA FISICA,BELEM,CAMPINA,2023,AGOSTO,01ª RISP,01ª AISP,2023-08-30,...,45,ADULTO IV (35 A 64 ANOS),PREJUDICADO,PREJUDICADO,ENSINO MEDIO COMPLETO,PREJUDICADO,M,66666.0,PREJUDICADO,EX-COMPANHEIRO(A)
2,FURTO,HIDROMETRO,SEM INSTRUMENTO,BELEM,MARACANGALHA,2023,OUTUBRO,01ª RISP,07ª AISP,2023-10-30,...,88888,"PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS",P,66666.0,NaN,NaN
3,FURTO,HIDROMETRO,SEM INSTRUMENTO,BELEM,UMARIZAL,2023,OUTUBRO,01ª RISP,01ª AISP,2023-10-28,...,88888,"PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS",P,66666.0,NaN,NaN
4,TRAFICO DE DROGAS,NaN,NaN,BELEM,TAPANA,2023,OUTUBRO,01ª RISP,11ª AISP,2023-10-19,...,88888,"PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS",P,66666.0,PREJUDICADO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69613,TRAFICO DE DROGAS,NaN,NaN,BELEM,CAMPINA,2023,SETEMBRO,01ª RISP,01ª AISP,2023-09-12,...,88888,"PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS",P,66666.0,PREJUDICADO,NaN
69614,TRAFICO DE DROGAS,NaN,NaN,BELEM,TAPANA,2023,SETEMBRO,01ª RISP,11ª AISP,2023-09-05,...,88888,"PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS",P,66666.0,PREJUDICADO,NaN
69615,ROUBO,EM RESIDENCIA/CELULAR/LEVE,ARMA DE FOGO,ANANINDEUA,CIDADE NOVA,2023,SETEMBRO,02ª RISP,18ª AISP,2023-09-01,...,62,ADULTO IV (35 A 64 ANOS),PREJUDICADO,PREJUDICADO,ENSINO MEDIO COMPLETO,SOLTEIRO(A),M,66666.0,PREJUDICADO,NaN
69616,TRAFICO DE DROGAS,NaN,NaN,ANANINDEUA,MAGUARI,2023,SETEMBRO,02ª RISP,20ª AISP,2023-09-05,...,88888,"PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS",P,66666.0,PREJUDICADO,NaN


In [ ]:
#df[df['MUNICÍPIO(S)']=='BELEM'][df['CONSOLIDADO(S)']=='ROUBO']

In [10]:
ab= df.groupby('CONSOLIDADO(S)')
ab.size().sort_values()

CONSOLIDADO(S)
MAUS TRATOS COM RESULTADO MORTE                  1
LESAO CORPORAL SEGUIDA DE MORTE                 12
LATROCINIO                                      12
MORTE NO TRANSITO                               61
HOMICIDIO NO TRANSITO                          156
MORTE POR INTERVENCAO DE AGENTE DO ESTADO      195
ESTUPRO                                        210
HOMICIDIO                                      341
ESTUPRO DE VULNERAVEL                          849
TRAFICO DE DROGAS                             1446
LESAO NO TRANSITO                             2739
LESAO CORPORAL                                6674
ROUBO                                        23871
FURTO                                        33051
dtype: int64

In [11]:
df[df['CONSOLIDADO(S)']=='ESTUPRO DE VULNERAVEL'].groupby('GRAU DE RELACIONAMENTO').size().sort_values()

GRAU DE RELACIONAMENTO
DOCENTE                       1
GENRO                         1
MAE                           1
TIO                           1
PROFESSOR(A)                  2
CUNHADO(A)                    2
EX-NAMORADO(A)                4
COMPANHEIRO(A)                5
EX-COMPANHEIRO(A)             5
IRMAO(A)                     10
AMIGO(A)                     11
PRIMO(A)                     20
PARENTESCO POR AFINIDADE     20
EX-PADRASTO                  23
NAMORADO(A)                  24
VIZINHO(A)                   26
DESCONHECIDO(A)              30
AVO                          52
CONHECIDO(A)                 52
TIO(A)                       64
PAI                          83
PADRASTO                    110
PREJUDICADO                 169
dtype: int64

In [112]:
df_vulneravel= df[df['CONSOLIDADO(S)']=='ESTUPRO']
df_vulneravel= pd.concat([df_vulneravel, df[df['CONSOLIDADO(S)']=='ESTUPRO DE VULNERAVEL']]).dropna(subset='GRAU DE RELACIONAMENTO')
#[df['GRAU DE RELACIONAMENTO'].isin(['PADRASTO', 'PAI', 'TIO(A)', 'AVO', 'VIZINHO(A)'])]

df_vulneravel= df_vulneravel[df_vulneravel['GRAU DE RELACIONAMENTO']!='PREJUDICADO']
df_vulneravel= df_vulneravel[df_vulneravel['FAIXA ETÁRIA VÍTIMA']!='PREJUDICADO']
df_vulneravel

,CONSOLIDADO(S),ESPECIFICAÇÃO CRIME,MEIO EMPREGADO DEAC,MUNICÍPIO(S),BAIRRO(S),ANO DO FATO,MÊS DO FATO,RISP,AISP,DATA DO FATO,...,IDADE VÍTIMA,FAIXA ETÁRIA VÍTIMA,TIPO DE VÍTIMA,COR VÍTIMA,GRAU DE INSTRUÇÃO VÍTIMA,ESTADO CIVÍL VÍTIMA,SEXO AUTOR,IDADE AUTOR,COR AUTOR,GRAU DE RELACIONAMENTO
1056,ESTUPRO,NaN,NaN,BELEM,GUAMA,2024,MAIO,01ª RISP,05ª AISP,2024-05-12,...,52,ADULTO IV (35 A 64 ANOS),PREJUDICADO,PREJUDICADO,ENSINO FUNDAMENTAL INCOMPLETO,SOLTEIRO(A),M,57.0,PREJUDICADO,EX-COMPANHEIRO(A)
1872,ESTUPRO,NaN,NaN,BELEM,CONDOR,2024,JUNHO,01ª RISP,03ª AISP,2024-03-06,...,19,ADULTO I (18 A 24 ANOS),PREJUDICADO,PREJUDICADO,ENSINO MEDIO COMPLETO,SOLTEIRO(A),M,66666.0,PREJUDICADO,VIZINHO(A)
3149,ESTUPRO,NaN,NaN,BELEM,MARACACUERA,2024,JUNHO,01ª RISP,12ª AISP,2024-06-06,...,14,ADOLESCENTE (12 A 17 ANOS),MENOR,PREJUDICADO,ENSINO FUNDAMENTAL COMPLETO,SOLTEIRO(A),M,18.0,PREJUDICADO,DESCONHECIDO(A)
3747,ESTUPRO,VIOLENCIA DOMESTICA,NaN,BELEM,TAPANA,2024,JUNHO,01ª RISP,11ª AISP,2024-06-06,...,14,ADOLESCENTE (12 A 17 ANOS),MENOR,PREJUDICADO,ENSINO FUNDAMENTAL INCOMPLETO,SOLTEIRO(A),M,30.0,PARDA,TIO(A)
3794,ESTUPRO,VIOLENCIA DOMESTICA,NaN,BELEM,FATIMA,2024,JUNHO,01ª RISP,02ª AISP,2024-06-03,...,28,ADULTO II (25 A 29 ANOS),PREJUDICADO,PREJUDICADO,ENSINO MEDIO COMPLETO,CASADO(A),M,27.0,PREJUDICADO,COMPANHEIRO(A)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68573,ESTUPRO DE VULNERAVEL,MENOR/VIOLENCIA DOMESTICA,NaN,ANANINDEUA,QUARENTA HORAS,2024,MAIO,02ª RISP,18ª AISP,2024-05-17,...,12,ADOLESCENTE (12 A 17 ANOS),MENOR,PREJUDICADO,ENSINO FUNDAMENTAL INCOMPLETO,SOLTEIRO(A),M,63.0,PREJUDICADO,PARENTESCO POR AFINIDADE
69370,ESTUPRO DE VULNERAVEL,MENOR/VIOLENCIA DOMESTICA,NaN,ANANINDEUA,QUARENTA HORAS,2024,MAIO,02ª RISP,18ª AISP,2024-05-18,...,9,CRIANCA (0 A 11 ANOS),MENOR,PREJUDICADO,ENSINO FUNDAMENTAL INCOMPLETO,SOLTEIRO(A),M,66666.0,PREJUDICADO,AVO
69465,ESTUPRO DE VULNERAVEL,MENOR,NaN,BELEM,TENONE,2024,MAIO,01ª RISP,13ª AISP,2024-05-24,...,11,CRIANCA (0 A 11 ANOS),MENOR,PREJUDICADO,ENSINO FUNDAMENTAL INCOMPLETO,SOLTEIRO(A),M,66666.0,PREJUDICADO,CONHECIDO(A)
69475,ESTUPRO DE VULNERAVEL,MENOR/VIOLENCIA DOMESTICA/ERA DO FATO,NaN,BELEM,MANGUEIRAO,2024,MAIO,01ª RISP,10ª AISP,2024-05-25,...,20,ADULTO I (18 A 24 ANOS),PREJUDICADO,PREJUDICADO,ENSINO MEDIO COMPLETO,UNIAO ESTAVEL,M,47.0,PREJUDICADO,PADRASTO


In [196]:
fig = px.sunburst(df_vulneravel, path=['FAIXA ETÁRIA VÍTIMA', 'GRAU DE RELACIONAMENTO' ] , title='Casos de estupro ocorridos no estado do Pará por grau de relacionamento', width=900, height=900)
fig.update_traces(
    hovertemplate='<b>%{label}</b><br>Contagem de casos: %{value} <extra></extra>')
annotations = [
    dict(
        x=-0.1, 
        y=-0.08, 
        text="Os maiores números de casos são entre vítimas que tem menos de 17 anos<br>Os maiores números de casos são com pessoas próximas e conhecidas", 
        font=dict(size=15, color="black"),
        showarrow=False,
        bgcolor="lightgrey",  # Fundo cinza claro
        align='left'
    ),
    # Adicione mais anotações conforme necessário
]

# Atualizar o layout para incluir as anotações
fig.update_layout(annotations=annotations, title_font=dict(size=20) )

fig.show()

In [ ]:
df[df['CONSOLIDADO(S)']=='FURTO'].groupby('MUNICÍPIO(S)').size().sort_values()

In [ ]:
bairro_maior_roubos= df[df['CONSOLIDADO(S)']=='ROUBO'][df['MUNICÍPIO(S)']=='BELEM'].groupby('BAIRRO(S)').size().sort_values().iloc[62:]
bairro_maior_trafico= df[df['CONSOLIDADO(S)']=='TRAFICO DE DROGAS'][df['MUNICÍPIO(S)']=='BELEM'].groupby('BAIRRO(S)').size().sort_values().iloc[57:]


In [4]:
bairro_maior_furto_belem= df[df['CONSOLIDADO(S)']=='FURTO'][df['MUNICÍPIO(S)']=='BELEM']
bairro_maior_furto_ananindeua= df[df['CONSOLIDADO(S)']=='FURTO'][df['MUNICÍPIO(S)']=='ANANINDEUA']
bairro_maior_furto_marituba= df[df['CONSOLIDADO(S)']=='FURTO'][df['MUNICÍPIO(S)']=='MARITUBA']
bairro_maior_furto_benevides= df[df['CONSOLIDADO(S)']=='FURTO'][df['MUNICÍPIO(S)']=='BENEVIDES']


C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_15388\230579388.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bairro_maior_furto_belem= df[df['CONSOLIDADO(S)']=='FURTO'][df['MUNICÍPIO(S)']=='BELEM']
C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_15388\230579388.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bairro_maior_furto_ananindeua= df[df['CONSOLIDADO(S)']=='FURTO'][df['MUNICÍPIO(S)']=='ANANINDEUA']
C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_15388\230579388.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bairro_maior_furto_marituba= df[df['CONSOLIDADO(S)']=='FURTO'][df['MUNICÍPIO(S)']=='MARITUBA']
C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_15388\230579388.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bairro_maior_furto_benevides= df[df['CONSOLIDADO(S)']=='FURTO'][df['MUNICÍPIO(S)']=='BENEVIDES']


In [7]:
#Belem
bairro_counts_belem = bairro_maior_furto_belem['BAIRRO(S)'].value_counts()
top_3_bairros_belem = bairro_counts_belem.head(3).index
top_3_data_belem = bairro_maior_furto_belem[bairro_maior_furto_belem['BAIRRO(S)'].isin(top_3_bairros_belem)]

#Ananindeua
bairro_counts_ananindeua = bairro_maior_furto_ananindeua['BAIRRO(S)'].value_counts()
top_3_bairros_ananindeua = bairro_counts_ananindeua.head(3).index
top_3_data_ananindeua = bairro_maior_furto_ananindeua[bairro_maior_furto_ananindeua['BAIRRO(S)'].isin(top_3_bairros_ananindeua)]

df_top= pd.concat([top_3_data_belem, top_3_data_ananindeua])

color_map = {
    'MARCO': '#636EFA',   # Substitua 'Bairro1_Belem' pelo nome real do bairro
    'PEDREIRA': '#EF553B',
    'CAMPINA': '#00CC96',
    'CIDADE NOVA': '#AB63FA',
    'CENTRO DE ANANINDEUA': '#19D3F3',
    'COQUEIRO ANANINDEUA': '#FFA15A'
}

app = Dash(__name__)


app.layout = html.Div([
    html.H4('Furtos por bairro em Belém e Ananindeua'),
    dcc.Graph(id="graph")
])


@app.callback(
    Output("graph", "figure"), 
    Input("graph", "id"))
def update_bar_chart(_):
    
    fig = px.bar(df_top, x='MUNICÍPIO(S)',color='BAIRRO(S)',
 barmode="group",height=400, color_discrete_map=color_map)
    print(df)
    return fig


app.run_server(debug=True)


          CONSOLIDADO(S)         ESPECIFICAÇÃO CRIME MEIO EMPREGADO DEAC  \
0      TRAFICO DE DROGAS                         NaN                 NaN   
1         LESAO CORPORAL                         NaN    VIOLENCIA FISICA   
2                  FURTO                  HIDROMETRO     SEM INSTRUMENTO   
3                  FURTO                  HIDROMETRO     SEM INSTRUMENTO   
4      TRAFICO DE DROGAS                         NaN                 NaN   
...                  ...                         ...                 ...   
69613  TRAFICO DE DROGAS                         NaN                 NaN   
69614  TRAFICO DE DROGAS                         NaN                 NaN   
69615              ROUBO  EM RESIDENCIA/CELULAR/LEVE        ARMA DE FOGO   
69616  TRAFICO DE DROGAS                         NaN                 NaN   
69617  TRAFICO DE DROGAS                         NaN                 NaN   

      MUNICÍPIO(S)     BAIRRO(S)  ANO DO FATO MÊS DO FATO      RISP      AISP  \
0     